# Example 28: Model-based control for cart-pole system

## Contents


* [Acknowledgements](#ackw)
* [Overview](#overview) 
    * [Model Predictive Control](#ekf)
    * [Summary of MPC](#sumekf)
    * [Motion Model](#motion_model)
    * [Observation Model](#observation_model)
* [Include files](#include_files)
* [The main function](#m_func)
* [Results](#results)
* [Source Code](#source_code)

## <a name="ackw"></a> Acknowledgements

This example is taken from <a href="https://github.com/forgi86/pyMPC">pyMPC</a>.

## <a name="overview"></a> Overview

In this notebook we will develop a model predictive controller, or MPC, for a cart-pole system. Concretely, the system we will assume is shown in the figure below



<figure>
<img src="cart_pole.png" alt="Cart-pole system"
	title="Cart-pole system" width="350" height="300" />
<figcaption>Figure: Cart-pole system. Image from <a href="https://github.com/forgi86/pyMPC">pyMPC</a></figcaption>
</figure>

## <a name="ekf"></a> MPC for cart-pole problem

The equations describing the motion of the cart-pole are

$$mL^2\frac{d^2\phi}{dt^2} + mL cos(\phi)\frac{d^2p}{dt^2} - mgLsin(\phi) = 0 $$

$$mLcos(\phi)\frac{d^2\phi}{dt^2} + (M + m)\frac{d^2p}{dt^2} - mgLsin(\phi)\frac{d^2\phi}{dt^2}  = F $$

with the following constraints

$$-\frac{\pi}{2} \leq \phi \leq \frac{\pi}{2}$$

$$-10 \leq p \leq 10$$

$$-100 \leq F \leq 100$$

Following a state-space represenation, we introduce the vector $\mathbf{x}$ as

$$\mathbf{x} = (x_1, x_2, x_3, x_4) = (\phi, p, \dot{\phi}, \dot{p})$$

MPC originated in the late seventies. The term MPC does not designate a specific control strategy but rather a range of control methods which make explicit use of a model of the process to obtain the control signal by minimizing an objective function. These design methods lead to controllers which have practically the same structure and present adequate degrees of freedom. The ideas, appearing in greater or lesser degree in the predictive control family, are basically:

- explicit use of a model to predict the process output at future time instants (horizon);

- calculation of a control sequence minimizing an objective function; and

- receding strategy, so that at each instant the horizon is displaced towards the future, which involves the application of the first control signal of the sequence calculated at each step.


In very simple terms, a MP controller uses a model of the system in order to make predictions about the system’s future behavior. The way this is done is by solving optimization problem in order to find the optimal control action that drives the predicted output to the reference. In general, the MPC framework  handle multi-input multi-output systems that may have interactions between their inputs and outputs. Furthermore, the framework  can also handle input and output constraints.

### Basic Ingredients

Let's review the basic ingredients needed in order to develop an MPC controller.

- Prediction model: needed in order to predict plant response to future input sequence
- Parameterization: of future input sequence into finite number of parameters
- Optimization: of future input sequence
- Reference trajectory
- Constraints

When we use a linear prediction model i.e. a model of the form
 

$$\mathbf{x}_{k+1} = \mathbf{A}_{k}\mathbf{x}_{k} + \mathbf{B}_{k}\mathbf{u}_{k}$$

then we have a linear MPC. On the other hand when the prediction model in non-linear then we deal with non-linear MPC. 

#### Assumptions

MPC overall uses the following assumptions:

- There is no mismatch between the plant model and prediction model
- No disturbance inputs
- All states are measured
- No sensor noise